<a href="https://colab.research.google.com/github/sohil344/Sklearn_Pipeline/blob/main/titanic_using_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv('/content/titanic_train.csv')


In [4]:
df.head()

,passenger_id,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,survived
0,1216,3,"Smyth, Miss. Julia",female,NaN,0,0,335432,7.7333,NaN,Q,13,NaN,NaN,1
1,699,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S,NaN,NaN,Croatia,0
2,1267,3,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",female,30.0,1,1,345773,24.1500,NaN,S,NaN,NaN,NaN,0
3,449,2,"Hocking, Mrs. Elizabeth (Eliza Needs)",female,54.0,1,3,29105,23.0000,NaN,S,4,NaN,"Cornwall / Akron, OH",1
4,576,2,"Veal, Mr. James",male,40.0,0,0,28221,13.0000,NaN,S,NaN,NaN,"Barre, Co Washington, VT",0


In [5]:
df=df.drop(columns=['passenger_id','name','cabin','boat','body','cabin','home.dest','ticket'])

In [9]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['survived']),
                                                 df['survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [10]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked
332,3,female,18.0,0,0,7.7750,S
383,1,female,58.0,0,0,146.5208,C
281,3,female,29.0,0,4,21.0750,S
2,3,female,30.0,1,1,24.1500,S
231,1,male,47.0,1,0,227.5250,C


In [11]:
y_train.sample(5)

521    0
501    1
188    0
425    0
568    0
Name: survived, dtype: int64

In [35]:
#imputation transformer

trf1=ColumnTransformer([('impute_age',SimpleImputer(),[2]),
                        ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6]),
                        ('impute_fare',SimpleImputer(),[5])],
                       remainder='passthrough')

In [36]:
#One Hot Encoding
trf2=ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])]
                       ,remainder='passthrough')

In [37]:
#Scaling
trf3=ColumnTransformer([('Scale',MinMaxScaler(),slice(0,10))
])

In [38]:
#Feature Selection
trf4=SelectKBest(score_func=chi2,k=8)


In [39]:
#Train Model
trf5=DecisionTreeClassifier()

In [40]:
pipe=Pipeline([
    ('trf1',trf1),('trf2',trf2),('trf3',trf3),('trf4',trf4),('trf5',trf5)
])

In [41]:
from sklearn import set_config
set_config(display='diagram')

In [42]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6]),
                                                 ('impute_fare',
                                                  SimpleImputer(), [5])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f561946cd30>)),
                ('trf5', DecisionTreeClassifier())])

In [43]:
y_pred = pipe.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6176470588235294

In [45]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6529411764705882

In [ ]:
params={'trf5_max'}